In [50]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout

In [51]:
# load numpy array from csv file
from numpy import loadtxt
# load array
X_train = loadtxt('x_train2.csv', delimiter=',')
Y_train = loadtxt('y_train2.csv', delimiter=',')
# print the array
X_train

array([[2., 6., 9., ..., 0., 0., 0.],
       [2., 6., 6., ..., 0., 0., 0.],
       [2., 6., 6., ..., 0., 0., 0.],
       ...,
       [2., 6., 6., ..., 0., 0., 0.],
       [2., 6., 6., ..., 0., 0., 0.],
       [2., 6., 9., ..., 0., 0., 0.]])

In [53]:
#reducing the size of the input length so it can train on a CPU
X_train = X_train[:, :100]
X_train

array([[ 2.,  6.,  9., ...,  4.,  7.,  4.],
       [ 2.,  6.,  6., ..., 10., 10., 21.],
       [ 2.,  6.,  6., ...,  4.,  5.,  4.],
       ...,
       [ 2.,  6.,  6., ...,  0.,  0.,  0.],
       [ 2.,  6.,  6., ...,  0.,  0.,  0.],
       [ 2.,  6.,  9., ..., 15.,  7.,  7.]])

In [54]:
Y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [55]:
VOCAB_SIZE = 1254
INPUT_LENGTH = 100 #1000
EMBEDDING_DIM = 128

In [56]:
from keras import backend as K
from keras.layers import Layer
from keras import initializers, regularizers, constraints

In [57]:
def dot_product(x, kernel):
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

In [58]:
#input_shape clarify
# keep reading

class AttentionWithContext(Layer):
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
#         assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)
        
    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)
        
        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
#         a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
#         print(K.sum(weighted_input, axis=1))
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [59]:
# model
def build_model(vocab_size, embedding_dim, input_length):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, input_length=input_length))
    model.add(SpatialDropout1D(0.2))
    model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
    model.add(AttentionWithContext())
    model.add(Dense(41, activation='softmax'))
    return model

In [60]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, INPUT_LENGTH)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 100, 128)          160512    
_________________________________________________________________
spatial_dropout1d_15 (Spatia (None, 100, 128)          0         
_________________________________________________________________
bidirectional_15 (Bidirectio (None, 100, 256)          263168    
_________________________________________________________________
attention_with_context_13 (A (None, 256)               66048     
_________________________________________________________________
dense_5 (Dense)              (None, 41)                10537     
Total params: 500,265
Trainable params: 500,265
Non-trainable params: 0
_________________________________________________________________
None


In [61]:
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 1800 samples, validate on 200 samples
Epoch 1/5
1800/1800 [==============================] - 31s 17ms/step - loss: 3.1404 - acc: 0.1406 - val_loss: 2.7666 - val_acc: 0.1800
Epoch 2/5
1800/1800 [==============================] - 20s 11ms/step - loss: 2.7363 - acc: 0.1533 - val_loss: 2.7796 - val_acc: 0.1250
Epoch 3/5
1800/1800 [==============================] - 20s 11ms/step - loss: 2.7055 - acc: 0.1500 - val_loss: 2.7273 - val_acc: 0.1950
Epoch 4/5
1800/1800 [==============================] - 19s 11ms/step - loss: 2.6777 - acc: 0.1650 - val_loss: 2.7335 - val_acc: 0.1650
Epoch 5/5
1800/1800 [==============================] - 19s 11ms/step - loss: 2.6408 - acc: 0.1739 - val_loss: 2.6783 - val_acc: 0.2050


In [8]:
example_x = X_train[0]
print(np.shape(example_x))
temp = model.predict(X_train[0:100])
# print(len(temp)), temp
print(temp[0])
for i in temp:
    print(np.argmax(i))


(1000,)
[0.00209911 0.00124006 0.00164237 0.21912028 0.00248105 0.20477231
 0.00102466 0.00090079 0.03958684 0.00195055 0.00073333 0.00081549
 0.00155815 0.05636364 0.00061747 0.02952985 0.00338738 0.00073309
 0.00321377 0.00821804 0.00102574 0.00211214 0.06001228 0.00204722
 0.08841579 0.00058595 0.0113609  0.00496192 0.02676554 0.00806735
 0.00112159 0.00996233 0.00418581 0.00158253 0.00446302 0.00129945
 0.0026114  0.1779126  0.00686241 0.00323006 0.00142568]
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
